In [1]:
# 📦 Step 1: Install Dependencies (Run this once)
!pip install crewai pandas matplotlib

# If not already done, install and run Ollama separately:
# 👉 https://ollama.com
# Then run a model like: `ollama run llama3`
import os

os.environ["LITELLM_API_BASE"] = "http://localhost:11434"
os.environ["LITELLM_MODEL"] = "llama3"
os.environ["MODEL"] = "ollama/llama3"  # Required for CrewAI to work with LiteLLM routing
os.environ["USE_LITELLM_ROUTER"] = "True"

In [2]:
import pandas as pd
from crewai import Agent, Task, Crew

class HealthTools:
    def __init__(self, df):
        self.df = df

    def get_health_summary(self):
        last_week = self.df[self.df["ActivityDate"] >= self.df["ActivityDate"].max() - pd.Timedelta(days=7)]
        avg_steps = last_week["TotalSteps"].mean()
        weight_change = last_week["WeightKg"].iloc[-1] - last_week["WeightKg"].iloc[0]
        return (
            f"In the last 7 days, you averaged {avg_steps:.0f} steps/day, "
            f"and your weight changed by {weight_change:.1f} lbs."
        )

    def suggest_nutrition(self):
        return "Try balanced meals with lean protein, greens, and avoid processed sugars."

    def set_goals(self):
        avg_steps = self.df["TotalSteps"].mean()
        return f"Set a goal of {int(avg_steps + 1000)} steps/day and minimum 7 hrs of sleep/night."


In [3]:
# Make sure your CSV is in the same folder or upload using the Jupyter UI
df = pd.read_csv("fitbit.csv", parse_dates=["ActivityDate"])
df.head()

,Unnamed: 0,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,...,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,0,1503960366,2016-04-05,14844,9.73,9.73,0.0,2.94,0.76,6.04,...,18,341,736,2083,53.299999,117.506384,22.0,22.969999,1.0,1.459901e+12
1,1,1927972279,2016-04-10,2105,1.46,1.46,0.0,0.00,0.00,1.46,...,0,139,805,2442,129.600006,285.719105,NaN,46.169998,0.0,1.460313e+12
2,2,2347167796,2016-04-03,7485,4.95,4.95,0.0,0.00,0.00,4.95,...,0,324,491,2035,63.400002,139.773078,10.0,24.770000,1.0,1.459728e+12
3,3,2873212765,2016-04-06,7350,4.94,4.94,0.0,0.00,0.00,4.94,...,0,382,1058,1910,56.700001,125.002104,NaN,21.450001,1.0,1.459987e+12
4,4,2873212765,2016-04-07,9353,6.31,6.31,0.0,0.37,0.69,5.24,...,17,363,1055,2035,57.200001,126.104416,NaN,21.650000,1.0,1.460074e+12


In [4]:
tools = HealthTools(df)

context_summary = tools.get_health_summary()
#sleep = tools.sleep_coaching()
nutrition = tools.suggest_nutrition()
goal = tools.set_goals()

user_health_context = (
    f"{context_summary}\n"
    #f"Sleep Insight: {sleep}\n"
    f"Nutrition Tip: {nutrition}\n"
    f"Goal Suggestion: {goal}\n"
)

print(user_health_context)


In the last 7 days, you averaged 10750 steps/day, and your weight changed by 32.5 lbs.
Nutrition Tip: Try balanced meals with lean protein, greens, and avoid processed sugars.
Goal Suggestion: Set a goal of 12442 steps/day and minimum 7 hrs of sleep/night.



In [5]:
health_agent = Agent(
    role="Health Coach",
    goal="Provide personalized health insights and advice based on Fitbit data",
    backstory="You are a wellness assistant helping users improve their health using their recent sleep, weight, and step data.",
    verbose=True,
    allow_delegation=False,
    llm="ollama/llama3"  # make sure ollama is running locally
)


In [6]:
print("👟 Welcome to your Health Assistant! Type your question (type 'exit' to quit)")

while True:
    user_input = input("🧑 You: ")
    if user_input.lower() == "exit":
        break

    prompt = (
        f"You are a health coach assistant. Based on the user's recent data:\n\n"
        f"{user_health_context}\n\n"
        f"Now respond to this question:\n"
        f"User: {user_input}\n"
        f"Assistant:"
    )

    task = Task(
        description=prompt,
        agent=health_agent,
        expected_output="A helpful and personalized response based on the user's health data and question.")
    crew = Crew(agents=[health_agent], tasks=[task], verbose=False)
    result = crew.kickoff()

    print("🤖 Assistant:", result)


👟 Welcome to your Health Assistant! Type your question (type 'exit' to quit)


🧑 You:  exit
